# Instalar um Core open5GS, o RIC O-RAN SC, uma gNB srsRAN e um UE srsRAN, e conectá-los.

#### Tutorial baseado em: https://docs.srsran.com/projects/project/en/latest/tutorials/source/near-rt-ric/source/index.html


### Instalar dependências
```bash
sudo apt-get install libzmq3-dev
sudo apt-get install libuhd-dev uhd-host
```

### Instalar o srsRAN:

```bash
cd ~
git clone https://github.com/srsran/srsRAN_Project.git
cd srsRAN_Project
mkdir build
cd build
cmake ../ -DENABLE_EXPORT=ON -DENABLE_ZEROMQ=ON
make -j`nproc`
```

Pay extra attention to the cmake console output. Make sure you read the following line:

```
-- FINDING ZEROMQ.
-- Checking for module 'ZeroMQ'
--   No package 'ZeroMQ' found
-- Found libZEROMQ: /usr/local/include, /usr/local/lib/libzmq.so
```

### Instalar o srsUE: 

```bash
sudo apt-get install build-essential cmake libfftw3-dev libmbedtls-dev libboost-program-options-dev libconfig++-dev libsctp-dev
cd ~
git clone https://github.com/srsRAN/srsRAN_4G.git
cd srsRAN_4G
mkdir build
cd build
cmake ../
make
make test
sudo make install
srsran_install_configs.sh user
```

### Instalar o O-RAN SC RIC
```bash
cd ~
git clone https://github.com/srsran/oran-sc-ric
```


### Baixar arquivos de configuração da gNB e do UE

```bash
wget https://docs.srsran.com/projects/project/en/latest/_downloads/cd7bc1de5ec01c261b2b112e21700e77/gnb_zmq.yaml ~/srsRAN_Project/build/apps/gnb/
wget https://docs.srsran.com/projects/project/en/latest/_downloads/fada80ca15bd69b34a0ad8425bbc6560/ue_zmq.conf ~/srsRAN_Project/build/apps/gnb/
```

## Rodar a rede

A seguintes etapas devem ser executadas em terminais separados.

1. Open5GS
2. NearRT-RIC
3. gNB
4. UE
5. Start IP traffic (e.g., ping)
6. xApp


### Open5GS Core

```bash
cd ./srsRAN_Project/docker
docker compose up 5gc
```

### O-RAN SC RIC

```bash
cd ~/oran-sc-ric
docker compose up
```

O Docker deve baixar e compilar (ao executar pela primeira vez) sete contêineres. Quando todos os contêineres forem implantados com sucesso, a seguinte mensagem deverá ser exibida na saída do console NearRT-RIC:

```
ric_submgr          | RMR is ready now ...
```

### gNB
```bash
cd ~/srsRAN_Project/build/apps/gnb/
sudo ./gnb -c gnb_zmq.yaml e2 --addr="10.0.2.10" --bind_addr="10.0.2.1"
```

A mensagem "Conectando ao AMF em 10.53.1.2:38412" indica que o gNB iniciou uma conexão com o núcleo. Já a mensagem "Conectando ao NearRT-RIC em 127.0.0.1:36421" indica que o gNB iniciou uma conexão com o NearRT-RIC.

Se a tentativa de conexão for bem-sucedida, o seguinte (ou similar) será exibido no console do NearRT-RIC:

```
ric_rtmgr_sim       | 2024/04/02 11:07:39 POST /ric/v1/handles/associate-ran-to-e2t  body: [{"E2TAddress":"10.0.2.10:38000","ranNamelist":["gnb_001_001_00019b"]}] elapsed: 10.77µs
```

### UE

```BASH
sudo ip netns add ue1
cd ~/srsRAN_4G/build/srsue/
sudo ./srsue ue_zmq.conf
```

Se o srsUE se conectar com sucesso à rede, o seguinte (ou similar) deverá ser exibido no console:

```
Built in Release mode using commit fa56836b1 on branch master.

Opening 1 channels in RF device=zmq with args=tx_port=tcp://127.0.0.1:2001,rx_port=tcp://127.0.0.1:2000,base_srate=11.52e6
Supported RF device list: UHD zmq file
CHx base_srate=11.52e6
Current sample rate is 1.92 MHz with a base rate of 11.52 MHz (x6 decimation)
CH0 rx_port=tcp://127.0.0.1:2000
CH0 tx_port=tcp://127.0.0.1:2001
Current sample rate is 11.52 MHz with a base rate of 11.52 MHz (x1 decimation)
Current sample rate is 11.52 MHz with a base rate of 11.52 MHz (x1 decimation)
Waiting PHY to initialize ... done!
Attaching UE...
Random Access Transmission: prach_occasion=0, preamble_index=0, ra-rnti=0x39, tti=334
Random Access Complete.     c-rnti=0x4601, ta=0
RRC Connected
PDU Session Establishment successful. IP: 10.45.1.2
RRC NR reconfiguration successful.
```

Fica claro que a conexão foi estabelecida com sucesso quando o UE recebe um IP, o que é visto em "Estabelecimento de Sessão PDU bem-sucedido". IP: 10.45.1.2. A conexão NR é então confirmada com a mensagem "Reconfiguração NR RRC bem-sucedida".

### Iniciar tráfego IP
```bash
sudo ip netns exec ue1 ping -i 0.1 10.45.1.1
```

Exemplo de saída de ping:
```
PING 10.45.1.1 (10.45.1.1) 56(84) bytes of data.
64 bytes from 10.45.1.1: icmp_seq=1 ttl=64 time=32.2 ms
64 bytes from 10.45.1.1: icmp_seq=2 ttl=64 time=35.3 ms
64 bytes from 10.45.1.1: icmp_seq=3 ttl=64 time=38.2 ms
64 bytes from 10.45.1.1: icmp_seq=4 ttl=64 time=71.5 ms
64 bytes from 10.45.1.1: icmp_seq=5 ttl=64 time=32.9 ms
```

Além disso, a ferramenta iperf pode ser usada para gerar tráfego com taxas de dados mais altas que o ping. Por exemplo, para enviar tráfego UL do UE, é necessário executar o seguinte comando:

```bash
sudo ip netns exec ue1 iperf -c 10.45.1.1 -u -b 10M -i 1 -t 60
```

### Iniciar xApp

Para iniciar o exemplo fornecido **kpm_mon_xapp.py**, execute o seguinte comando no diretório oran-sc-ric:

```bash
cd ~/oran-sc-ric
docker compose exec python_xapp_runner ./kpm_mon_xapp.py --metrics=DRB.UEThpDl,DRB.UEThpUl --kpm_report_style=5
```

O xApp permite a assinatura de todos os Estilos de Relatório do E2SM-KPM (ou seja, 1 a 5) e a definição dos nomes das métricas. Com os parâmetros acima, o xApp deve assinar as medições DRB.UEThpUl e DRB.UEThpUl e exibir o conteúdo das mensagens RIC_INDICATION recebidas. A saída do console do xApp deve ser semelhante a:

```
RIC Indication Received from gnb_001_001_00019b for Subscription ID: 5, KPM Report Style: 5
E2SM_KPM RIC Indication Content:
-ColletStartTime:  2024-04-02 13:24:56
-Measurements Data:
--UE_id: 0
---granulPeriod: 1000
---Metric: DRB.UEThpDl, Value: [7]
---Metric: DRB.UEThpUl, Value: [7]
```
